# Customer Service Query Classification 

### Import libaries

In [ ]:
#!pip install missingno

In [1]:
import pandas as pd
import missingno
import matplotlib.pyplot as plt
import seaborn as sns

### Load data

In [2]:
df = pd.read_csv("data/Customer_Service_Questions_Multiclass.csv")

### Initial data preparation

In [ ]:
# check for missing values
# missingno.matrix(df) # there are no missing values

In [3]:
'''
# distribution of the topics column
sns.countplot(x=df["topic"])
plt.xlabel("Topic")
plt.ylabel("Frequency")
plt.xticks(rotation=65)
'''

'\n# distribution of the topics column\nsns.countplot(x=df["topic"])\nplt.xlabel("Topic")\nplt.ylabel("Frequency")\nplt.xticks(rotation=65)\n'

In [4]:
# encoding the labels
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['category'] = encoder.fit_transform(df['topic'])

In [5]:
# train-test split
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(df.question, df.topic, stratify=df.topic, test_size=0.2, random_state=42)

### Build, Train, and Evaluate Model

Models to consider:
-
1. One multi-class classifier (e.g., Naive Bayes, Logistic, Decision Tree, SVM)
2. One ensemble classifier whose code is also provided (e.g., Random Forest, XGBoost)
3. One other model of your choice whose code is NOT provided in class handouts

Input features to consider for each model:
-
1. TF-IDF vector of tokenized words
2. TF-IDF vector of n-grams (of range 4-5)
3. Word vectors (Glove, Word2Vec, or FastText)
4. Document vectors (Doc2Vec)

In [6]:
# TF-IDF vector
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(df.question)

dfTfidf_train = vectorizer.transform(xtrain)
dfTfidf_test = vectorizer.transform(xtest)

# TF-IDF vector of n-grams
#dfTfidf_ngrams = 

# Word vectors 
#dfWordVec = 

# TF-IDF vector
#dfDocVec = 

#### Build model

In [7]:
# svc
from sklearn.svm import SVC
svc = SVC()

#### Train model and make predictions

In [8]:
model = svc.fit(dfTfidf_train, ytrain)

tfidf_pred = model.predict(dfTfidf_test)

#### Evaluate model

In [9]:
from sklearn.metrics import classification_report
print(classification_report(ytest, tfidf_pred))

                        precision    recall  f1-score   support

           Omnichannel       1.00      0.99      0.99        90
  Product Availability       0.98      0.99      0.99       167
    Product Comparison       1.00      0.94      0.97       161
Product Specifications       0.91      0.99      0.95       168
     Returns & Refunds       0.99      1.00      1.00       153
      Sales/Promotions       1.00      0.92      0.96       101
              Shipping       0.99      1.00      1.00       160

              accuracy                           0.98      1000
             macro avg       0.98      0.98      0.98      1000
          weighted avg       0.98      0.98      0.98      1000



### Hyperparameter optimization

### Deploy model?